In [ ]:
# Import libraries and loading the csv file
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('omw-1.4')
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
vehicles_clean = pd.read_csv(r"C:\Users\91886\Downloads\vehicles_allclean_imputer.csv")

In [ ]:
vehicles_clean.info()

In [ ]:
vehicles_clean.isna().sum()

In [ ]:
vehicles_clean.drop(['condition','id','posting_date','model','description'], axis=1,inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# define the features and target variables
X = vehicles_clean.drop('price', axis=1)
y = vehicles_clean['price']

# divide the data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# display the shapes of the resulting datasets
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")

In [ ]:
type(X_train)

In [ ]:
type(X_val)

In [ ]:
X_train.dtypes

In [ ]:
cat_cols = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'paint_color', 'state', 'drive']
num_cols = ['year','cylinders','odometer','lat','long','price']
# define the pipeline to perform one-hot encoding
ohe_pipeline = Pipeline([
    ('ohe_transformer', ColumnTransformer(
        transformers=[
            ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols)
        ],
        remainder='passthrough'
    ))
])

In [ ]:
X_train = ohe_pipeline.fit_transform(X_train)
X_val = ohe_pipeline.transform(X_val)
X_test = ohe_pipeline.transform(X_test)

In [ ]:
# class DataFrameSelector(BaseEstimator, TransformerMixin):
#     def __init__(self, feature_names):
#         self.attribute_names = feature_names
#     def fit(self, X, y=None):
#         return self
#     def transform(self, X):
#         return X[self.attribute_names].values

In [ ]:
# # Impute fills missing numerals by the median of the remaining data
# imputer = SimpleImputer(strategy="median")

# # The imputer is fit on the training data. It can then also be applied to the test data (without a refit)
# num_pipeline = Pipeline([
#         ('selector', DataFrameSelector(num_cols)),
#         ('imputer', SimpleImputer(strategy="median")),
#         ('std_scaler', StandardScaler())
#     ])

In [ ]:
# X_train = num_pipeline.fit_transform(X_train)

In [ ]:
type(X_val)

# Linear Regression

In [ ]:
linreg_model = LinearRegression()
linreg_model.fit(X_train, y_train)

In [ ]:
y_val_pred = linreg_model.predict(X_val)

In [ ]:
mse_val = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)

In [ ]:
print('Mean squared error on validation data:', np.sqrt(mse_val))
print("R-squared:", r2)

In [ ]:
y_val_pred = pd.DataFrame(y_val_pred, columns = ['Predicted Output'])
lin_reg_results = pd.concat([y_val_pred, y_val.to_frame().reset_index(drop = True)], axis = 1, ignore_index = False) 

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = lin_reg_results, y = 'Predicted Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

In [ ]:
#Lin Reg on log transformed target variable

In [ ]:
# Log transform the target variable
y_train_log = np.log(y_train)

In [ ]:
# Fit the model to the log-transformed target variable
linreg_model.fit(X_train, y_train_log)

In [ ]:
# Predict the log-transformed car prices for the validation data
y_pred_log = linreg_model.predict(X_val)

In [ ]:
# Transform the predicted values back to the original scale
y_pred = np.exp(y_pred_log)

In [ ]:
# Evaluate the model's performance on the validation data
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

In [ ]:
print('Mean squared error on validation data:', mse_val)
print("R-squared:", r2)

In [ ]:
y_pred = pd.DataFrame(y_pred, columns = ['Predicted Output'])
lin_reg_results = pd.concat([y_pred, y_val.to_frame().reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = lin_reg_results, y = 'Predicted Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

# Decision Tree

In [ ]:
dtr_model = DecisionTreeRegressor(splitter = 'random')
dtr_model.fit(X_train, y_train)

In [ ]:
dtr_predict_train = dtr_model.predict(X_train)

In [ ]:
dtr_predict_val = dtr_model.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, dtr_predict_train)) 
r2_train = r2_score(y_train, dtr_predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, dtr_predict_val)) 
r2_val = r2_score(y_val, dtr_predict_val)
print(rmse_val)
print(r2_val)

In [ ]:
dtr_predict_val = pd.DataFrame(dtr_predict_val, columns = ['Predicted Val Output'])

In [ ]:
dtr_results = pd.concat([dtr_predict_val, y_val.to_frame().reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = dtr_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

In [ ]:
#Decision Treee on log transformed target variable

In [ ]:
# Log transform the target variable
y_train_log = np.log(y_train)
# y_val_log = np.log(y_val)

In [ ]:
# Fit the model to the log-transformed target variable
dtr_model.fit(X_train, y_train_log)

In [ ]:
dtr_predict_train_log = dtr_model.predict(X_train)

In [ ]:
dtr_predict_val_log = dtr_model.predict(X_val)

In [ ]:
predict_train = np.exp(dtr_predict_train_log)

In [ ]:
predict_val = np.exp(dtr_predict_val_log)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, predict_train)) 
r2_train = r2_score(y_train, predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, predict_val)) 
r2_val = r2_score(y_val, predict_val)
print(rmse_val)
print(r2_val)

# W/o log data 
# 6124.774719287382
# 0.8253020280442003

In [ ]:
dtr_predict_val = pd.DataFrame(predict_val, columns = ['Predicted Val Output'])

In [ ]:
dtr_results = pd.concat([dtr_predict_val, y_val.to_frame().
                         reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = dtr_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

# Random Forest

In [ ]:
# Create a random forest regressor with 100 trees
rfr_model = RandomForestRegressor(n_estimators=500, max_depth=10, bootstrap=True, random_state=42)

# max_depth -> 10 

In [ ]:
# fit the model to the training data
rfr_model.fit(X_train, y_train)

In [ ]:
rfr_predict_train = rfr_model.predict(X_train)
rfr_predict_val = rfr_model.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, rfr_predict_train)) 
r2_train = r2_score(y_train, rfr_predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, rfr_predict_val)) 
r2_val = r2_score(y_val, rfr_predict_val)
print(rmse_val)
print(r2_val)

In [ ]:
rfr_predict_val = pd.DataFrame(rfr_predict_val, columns = ['Predicted Val Output'])
rfr_results = pd.concat([rfr_predict_val, y_val.to_frame().
                         reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = rfr_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

In [ ]:
# On Log transformed 

In [ ]:
# Log transform the target variable
y_train_log = np.log(y_train)
# y_val_log = np.log(y_val)

In [ ]:
# Create a random forest regressor with 100 trees
rfr_model_log = RandomForestRegressor(n_estimators=500, max_depth=10, bootstrap=True, random_state=42)

# max_depth -> 10 

In [ ]:
rfr_model_log.fit(X_train, y_train_log)

In [ ]:
rfr_predict_train_log = rfr_model_log.predict(X_train)
rfr_predict_val_log = rfr_model_log.predict(X_val)

In [ ]:
predict_train = np.exp(rfr_predict_train_log)
predict_val = np.exp(rfr_predict_val_log)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, predict_train)) 
r2_train = r2_score(y_train, predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, predict_val)) 
r2_val = r2_score(y_val, predict_val)
print(rmse_val)
print(r2_val)

In [ ]:
rfr_results = pd.concat([rfr_predict_val, y_val.to_frame().
                         reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = rfr_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

# XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb_model = XGBRegressor()

In [ ]:
# fit the model to the training data
xgb_model.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model.predict(X_train)
xgb_predict_val = xgb_model.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

In [ ]:
xgb_predict_val = pd.DataFrame(xgb_predict_val, columns = ['Predicted Val Output'])
xgb_results = pd.concat([xgb_predict_val, y_val.to_frame().
                         reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = xgb_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

In [ ]:
# Log transform the target variable
y_train_log = np.log(y_train)
# y_val_log = np.log(y_val)

In [ ]:
xgb_model.fit(X_train, y_train_log)

In [ ]:
xgb_predict_train_log = xgb_model.predict(X_train)
xgb_predict_val_log = xgb_model.predict(X_val)

In [ ]:
predict_train = np.exp(xgb_predict_train_log)
predict_val = np.exp(xgb_predict_val_log)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, predict_train)) 
r2_train = r2_score(y_train, predict_train)
print(rmse_train)
print(r2_train)

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, predict_val)) 
r2_val = r2_score(y_val, predict_val)
print(rmse_val)
print(r2_val)

In [ ]:
predict_val = pd.DataFrame(predict_val, columns = ['Predicted Val Output'])
xgb_results = pd.concat([predict_val, y_val.to_frame().
                         reset_index(drop = True)], axis = 1, ignore_index = False)

In [ ]:
plt.figure(figsize = (10, 10))
sns.regplot(data = xgb_results, y = 'Predicted Val Output', x = 'price', color = 'coral', marker = 'o')
plt.title("Comparision of predicted values and the actual values", fontsize = 20)
plt.show()

In [ ]:
# Hyperparameters 
params = {'max_depth': [5,8,10],
          'learning_rate': [0.01, 0.05, 0.1],
          'n_estimators': [100, 500, 1000],
          'gamma': [0, 0.2, 0.4],
          'reg_alpha': [0, 0.5, 5],
          'reg_lambda': [1, 10, 100]
          }
# params = {
#     'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
#     'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
#     'n_estimators': [100, 500, 1000, 2000, 3000],
#     'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7],
#     'gamma': [0, 0.1, 0.2, 0.3, 0.4],
#     'subsample': [0.5, 0.6, 0.7, 0.8, 0.9],
#     'reg_alpha': [0, 0.1, 0.5, 1, 10],
#     'reg_lambda': [0.01, 0.1, 1, 10, 100]
# }

In [ ]:
# Define the randomized search
random_search = RandomizedSearchCV(
    xgb_model, param_distributions=params,
    n_iter=50, cv=5, verbose=1, n_jobs=-1)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
# DEFAULT ONE 

# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
#              gamma=0, gpu_id=-1, importance_type=None,
#              interaction_constraints='', learning_rate=0.300000012,
#              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
#              monotone_constraints='()', n_estimators=100, n_jobs=4,
#              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
#              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
#              validate_parameters=1, verbosity=None)

In [ ]:
# HYPERPARAMETER 1 --> Overfitting observed 

xgb_model1 = XGBRegressor(learning_rate = 0.5, n_estimators=500,max_depth=10)

In [ ]:
xgb_model1.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model1.predict(X_train)
xgb_predict_val = xgb_model1.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# HYPERPARAMETER 2 
# (with regularization parameters)
# reduced learning rate
# max depeth reduced

## overfitting 

xgb_model2 = XGBRegressor(learning_rate = 0.3, n_estimators=500,max_depth=8, reg_alpha=0.01, reg_lambda=1)

In [ ]:
xgb_model2.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model2.predict(X_train)
xgb_predict_val = xgb_model2.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# HYPERPARAMETER 3


xgb_model3 = XGBRegressor(learning_rate = 0.4, n_estimators=500,max_depth=6, reg_alpha=0.05, reg_lambda=1)

In [ ]:
xgb_model3.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model3.predict(X_train)
xgb_predict_val = xgb_model3.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# HYPERPARAMETER 4
#. increase reg_alpha

xgb_model4 = XGBRegressor(learning_rate = 0.4, n_estimators=500,max_depth=6, reg_alpha=0.1, reg_lambda=1)

In [ ]:
xgb_model4.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model4.predict(X_train)
xgb_predict_val = xgb_model4.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
#              gamma=0, gpu_id=-1, importance_type=None,
#              interaction_constraints='', learning_rate=0.300000012,
#              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
#              monotone_constraints='()', n_estimators=100, n_jobs=4,
#              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
#              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
#              validate_parameters=1, verbosity=None)

In [ ]:
# HYPERPARAMETER 5
#. n_estimators

xgb_model5 = XGBRegressor(learning_rate = 0.4, n_estimators=200,max_depth=6, reg_alpha=0.1, reg_lambda=1)

In [ ]:
xgb_model5.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model5.predict(X_train)
xgb_predict_val = xgb_model5.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# HYPERPARAMETER 6


xgb_model6 = XGBRegressor(max_depth=8,learning_rate = 0.5)
xgb_model6.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model6.predict(X_train)
xgb_predict_val = xgb_model6.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# HYPERPARAMETER 7
xgb_model7 = XGBRegressor(learning_rate = 0.38, n_estimators=150,max_depth=8)
xgb_model7.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model7.predict(X_train)
xgb_predict_val = xgb_model7.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# Hyperparameter 8
#  max_depth is set to 6 and min_child_weight is set to 5, 

xgb_model8 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=5,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
xgb_model8.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model8.predict(X_train)
xgb_predict_val = xgb_model8.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# Hyperparameter 9
# learning rate increased  

xgb_model9 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.35,
             max_delta_step=0, max_depth=6, min_child_weight=5,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
xgb_model9.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model9.predict(X_train)
xgb_predict_val = xgb_model9.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413

In [ ]:
# Hyperparameter 10
# max_depth increased  

xgb_model10 = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.35,
             max_delta_step=0, max_depth=8, min_child_weight=5,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)
xgb_model10.fit(X_train, y_train)

In [ ]:
xgb_predict_train = xgb_model10.predict(X_train)
xgb_predict_val = xgb_model10.predict(X_val)

In [ ]:
rmse_train = sqrt(mean_squared_error(y_train, xgb_predict_train)) 
r2_train = r2_score(y_train, xgb_predict_train)
print(rmse_train)
print(r2_train)

# 4706.4166370845305
# 0.8950213517845189

In [ ]:
rmse_val = sqrt(mean_squared_error(y_val, xgb_predict_val)) 
r2_val = r2_score(y_val, xgb_predict_val)
print(rmse_val)
print(r2_val)

# 5530.851800951771
# 0.8575404171042413